<center><img src="https://storage.googleapis.com/arize-assets/arize-logo-white.jpg" width="200"/></center>

# Query For All Custom Metrics Using GraphQL API

This example walks through how to query for all custom metrics in a given Space and export to a CSV file. For more information on the GraphQL API for custom metrics, see the Arize documentation [here](https://docs.arize.com/arize/api-reference/graphql-api/custom-metrics-api).

## Install and Import Dependencies

In [ ]:
%pip install -q gql[all]
import time
import pandas as pd
from gql import Client, gql
from gql.transport.requests import RequestsHTTPTransport
from gql.transport.exceptions import TransportQueryError

## Get your API key
First, make sure you have developer permissions. If you are able to visit the [API explorer](https://app.arize.com/graphql), then you have developer permissions. If not, please ask your Account Admin to provide you with access.

The API key can be retrieved from the [API explorer](https://app.arize.com/graphql) page. Click the button on the top right called "Get Your API Key." A modal will pop up with your key, copy that into the `API_KEY` constant below.

NOTE: this key is different than the SDK key used to send data to Arize.

In [ ]:
API_KEY = "API_KEY"

# Select your transport with a defined URL endpoint
transport = RequestsHTTPTransport(
    url="https://app.arize.com/graphql/", headers={"x-api-key": API_KEY}
)

# Create a GraphQL client using the defined transport
client = Client(transport=transport, fetch_schema_from_transport=True)

## Get all model IDs in a given space

In [ ]:
# We start this query from your space. Spaces have globally unique IDs. You can get your space ID by visiting app.arize.com.
# The url will be in this format: https://app.arize.com/organizations/:orgId/spaces/:spaceId
# NOTE: this is not the same as the space key used to send data using the SDK
SPACE_ID = "SPACE_ID"

In [ ]:
# Function for getting all your models in the space
def get_models(space_id):
    # A re-usable query for fetching your models, a page at a time
    models_query = gql(
    """
        query getModels($spaceId: ID!, $cursor: String) {
            space: node(id: $spaceId) {
                ... on Space {
                    name
                    models (first: 50, after: $cursor) {
                        pageInfo {
                            endCursor
                        }
                        edges {
                            model: node {
                                id
                                name
                            }
                        }
                    }
                }
            }
        }
    """
    )
    # Base query parameters for fetching models
    params = {"spaceId": space_id}

    # An array of models that we will append to
    models = []
    space_name = ""

    # Execute the query on the transport. Continue to pull data until there is no more monitors
    print("Retrieving models...")
    while True:
        paged_response = client.execute(models_query, params)
        space_name = paged_response["space"]["name"]
        # Append the monitors to your list
        models.extend(paged_response["space"]["models"]["edges"])
        # If there is another page of information, point the cursor to the next page and fetch more
        end_cursor = paged_response["space"]["models"]['pageInfo']['endCursor']
        print('pageInfo end_cursor %s' % (end_cursor))
        if end_cursor:
            print("There is another page of models. Loading more.")
            params["cursor"] = end_cursor
        else:
            # No more models to pull. The list is complete!
            break
    print("\nRetrieved {} models.".format(len(models)))
    return models

In [ ]:
# Get all models in the space
models = get_models(SPACE_ID)

In [ ]:
# Put models into dataframe
models_df = pd.json_normalize(models, sep=".")
models_df.head()

## Get custom metric IDs for each model

In [ ]:
# Function get IDs for each custom metric
def fetch_custom_metric_ids(model_id):
    custom_metric_query = gql("""
        query GetCustomMetricIDs($modelId: ID!) {
            node(id: $modelId) {
                ...on Model {
                    customMetrics(first: 10) {
                        edges {
                            node {
                                id
                            }
                        }
                    }
                }
            }
        }
    """)
    response = client.execute(custom_metric_query, {"modelId": model_id})
    metric_ids = [edge['node']['id'] for edge in response['node']['customMetrics']['edges']]
    return metric_ids

models_df['custom_metric_ids'] = models_df['model.id'].apply(fetch_custom_metric_ids)

In [ ]:
models_df

## Get custom metric config for each custom metric ID

In [ ]:
def fetch_custom_metric_meta(metric_id):
    metric_meta_query = gql("""
        {
            node(id: "$metricId") {
                ... on CustomMetric {
                    id
                    createdAt
                    name
                    description
                    metric
                    requiresPositiveClass
                }
            }
        }
    """.replace("$metricId", metric_id))  # Using replace for simplicity in this example
    response = client.execute(metric_meta_query)
    return response['node']


def fetch_custom_metric_meta_with_retry(metric_id, retry_delay=5, max_retries=5):
    """Fetch custom metric meta data with retries on rate limit errors."""
    for attempt in range(max_retries):
        try:
            # Attempt to fetch the custom metric meta data
            return fetch_custom_metric_meta(metric_id)
        except TransportQueryError as e:
            if "TOO_MANY_REQUESTS" in str(e):
                print(f"Rate limit hit, retrying in {retry_delay} seconds... (Attempt {attempt + 1}/{max_retries})")
                time.sleep(retry_delay)
                # Optionally, increase retry_delay here for exponential backoff
            else:
                # If the error is not related to rate limiting, raise it
                raise e
    raise Exception(f"Failed to fetch custom metric meta after {max_retries} attempts due to rate limiting.")


# Flatten the list of lists of custom metric IDs
all_metric_ids = [metric_id for sublist in models_df['custom_metric_ids'].tolist() for metric_id in sublist]

# Fetch metadata for each metric ID
metrics_meta_data = [fetch_custom_metric_meta_with_retry(metric_id) for metric_id in all_metric_ids]

# # Convert to DataFrame
metrics_meta_df = pd.DataFrame(metrics_meta_data)

In [ ]:
metrics_meta_df